In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca
from decision_vars import DecisionVarSet


In [2]:
# Test decision vars
from decision_vars import DecisionVarSet
a = DecisionVarSet({'x':[1.0, 2.0]}, lb={})
print(a)
b = DecisionVarSet({'y':2.0}, lb={'y':3.0})
a += b
print(a.vectorize('lb'))
d = a.dictize(np.array([0.1, 0.3, 0.5]))
print(d)

***** Decision Vars *****
Attributes: ['sym', 'init', 'lb']
Vars: 
  x: [x_0, x_1], (2, 1)

[-inf, -inf, 3]
{'x': array([0.1, 0.3]), 'y': array(0.5)}


In [4]:
from robot import Robot
rob = Robot('franka.urdf')

Building robot model from franka.urdf with TCP fr3_link8
***** Decision Vars *****
Attributes: ['sym', 'init', 'cov', 'noise']
Vars: 
  q: [q_0, q_1, q_2, q_3, q_4, q_5, q_6], (7, 1)
  dq: [dq_0, dq_1, dq_2, dq_3, dq_4, dq_5, dq_6], (7, 1)



In [5]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [10, 10, 10],
        'rest':[0, 0, 0.5]}
c = Contact(pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
c.F_fn(ca.DM([0.5, 0.3, -0.5]), ca.DM.ones(3))

c2 = Contact(pars, ['stiff'], {'cov':{}, 'noise':{}})
print(c2.F_fn(ca.DM([0.5, 0.3, -0.5]), ca.DM.ones(3), ca.DM([10, 0, 0])))

[-5, -0, 0]


In [11]:
from robot import Robot
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [10, 10, 10],
        'rest':[0, 0, 0.5]}
c2 = Contact(pars, ['stiff'], {'cov':{}, 'noise':{}})
rob = Robot('franka.urdf', subsys = [c2])
rob.build_disc_dyn(0.1)

Building robot model from franka.urdf with TCP fr3_link8
  with 1 subsys
***** Decision Vars *****
Attributes: ['sym', 'init', 'cov', 'noise']
Vars: 
  q: [q_0, q_1, q_2, q_3, q_4, q_5, q_6], (7, 1)
  dq: [dq_0, dq_1, dq_2, dq_3, dq_4, dq_5, dq_6], (7, 1)
  stiff: [stiff_0, stiff_1, stiff_2], (3, 1)

(3, 1)
(3, 7)


KeyError: 'xi'